In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import Quandl
import math

def retrieveStockCodes():
    dow_jones_codes = pd.read_csv("C:/Users/JohnSmith2/Downloads/dowjonesA.csv")
    dj_df = pd.DataFrame(dow_jones_codes)
    dj_df = dj_df.ix[:,['name','free_code']]
    return dj_df
    
def retrieveDowJonesData(dj_df):
    dowJonesDict = {}
    for name, value in zip(dj_df['name'], dj_df['free_code']):
        each = Quandl.get(value, trim_start="2010-04-15", trim_end="2016-04-15", authtoken="gEC9xAKi4avigPpoQPX1",
                         returns='numpy')
        df = pd.DataFrame(each)
        
        #fill forward and backword missing data
        df = df.fillna(method='ffill', inplace=True)
        df = df.fillna(method='bfill', inplace=True)
        
        dowJonesDict[name] = pd.DataFrame(df)
    return dowJonesDict

dj_df = retrieveStockCodes()
dowJonesDict = retrieveDowJonesData(dj_df)
print dowJonesDict.keys()

['Johnson & Johnson', 'IBM', 'United Technologies', 'Nike', 'General Electric', 'The Home Depot', 'Verizon', "McDonald's", 'American Express', 'Coca-Cola', 'Caterpillar', 'Visa', 'Walt Disney', 'Boeing', 'Intel', 'Pfizer', 'Procter & Gamble', 'Wal-Mart', 'Merck', 'UnitedHealth Group', 'Cisco Systems', '3M', 'Microsoft', 'JPMorgan Chase', 'Apple', 'Travelers', 'DuPont', 'Goldman Sachs', 'Chevron', 'ExxonMobil']


In [2]:
def get_rolling_mean(values, window):
    """Return rolling mean of given values, using specified window size."""
    return pd.rolling_mean(values, window=window)
    
def get_rolling_std(values, window):
    """Return rolling standard deviation of given values, using specified window size"""
    return pd.rolling_std(values, window=window)
    
def get_bollinger_bands(rm, rstd):
    """Return upper and lower Bollinger Bands."""
    upper_band = rm + rstd * 2
    lower_band = rm - rstd * 2
    return upper_band, lower_band
    
def compute_daily_returns(df):
    """Compute and return the daily return values."""
    daily_returns = (df / df.shift(1)) - 1
    daily_returns.ix[0,:] = 0 #Pandas leaves the 0th row full of NaNs
    return daily_returns
    
def plot_data(df, title="Stock prices", xlabel="Date", ylabel="Price"):
    """Plot stock prices with a custom title and meaningful axis labels."""
    ax = df.plot(title=title, fontsize=12)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    plt.show()
    
def createAdjVolAndCloseDfs(dowJonesDict):
    i = 0
    for each in dowJonesDict.keys():
        if i == 0:
            df = dowJonesDict[each]
            adj_close_df = df[['Date','Adj. Close']].rename(columns={'Adj. Close': each})
            adj_vol_df = df[['Date','Adj. Volume']].rename(columns={'Adj. Volume': each})
            i = 1
        else:
            df = dowJonesDict[each]
            next_adj_close_df = df[['Date','Adj. Close']].rename(columns={'Adj. Close': each})
            next_adj_vol_df = df[['Date','Adj. Volume']].rename(columns={'Adj. Volume': each})
            adj_close_df = pd.merge(adj_close_df, next_adj_close_df, on='Date')
            adj_vol_df = pd.merge(adj_vol_df, next_adj_vol_df, on='Date')
            
    return adj_close_df, adj_vol_df

In [3]:
def daily_return_stats(daily_returns):
    #get mean and std deviation of daily returns
    daily_returns.hist()
    beta_XOM,alpha_XOM = np.polyfit(daily_returns['SPY'],daily_returns['XOM'],1)
    # Calculate correlation coefficient
    print daily_returns.corr(method='pearson')

def daily_portfolio_values(adj_close_df, adj_vol_df, daily_returns):
    # Daily portfolio value
    
    # TODO: 
    normalize_prices = prices/prices[0]
    allocations = normalized_prices * adj_vol_df #allocs
    position_vals = allocations * start_val
    portfolio_val = position_vals.sum(axis=1)
    
    # Portfolio stats
    daily_rets = daily_returns[1:]
    cum_ret = (portfolio_val[-1]/portfolio_val[0]) - 1
    avg_daily_ret = daily_rets.mean()
    std_daily_ret = daily_rets.std()
    
    #Definition of portfolio:
    #  Wi: portion of funds in asset i
    #Cap weighted:
    #  Wi = mktcap i / sum(mktcap all)
    Rp(t) = sum(Wi*Ri(t))

def sharpe_ratio(daily_returns):
    # Sharpe ratio == risk adjusted return
    # lower risk, higher return is better
    # considers risk free rate of return too
    # SR is an annual measure
    var = 1.0 + var
    s = newton_root(252, var)
    daily_rf = s - 1
    sharpe_ratio = mean(daily_returns - daily_rf)/std(daily_returns)
    K = sqrt(252)    #weekly K = sqrt(52), etc.
    SRannualized = K * sharpe_ratio
    
def newton_root(k, n):
    k = 252
    n = 1 + var
    u, s = n, n+1
    while u < s:
        s = u
        t = (k-1) * s + n // pow(s, k-1)
        u = t // k
    return s

def forcasting_algorithm():
    # Forcasting algorithm inputs:
    # -Information feed
    # -Historical price data
    intrinsic_value = future_value / discount_rate #(ie. if dr=5%,fv=1, 1/.05 = 20$)
    book_value = total_assets - (intangibal_assets + liabalities)
    market_capitalization = num_shares * price

def capm_equation():
    #****CAPM EQUATION****
    # Ri(t) == return for particular stock on a given day t
    # Bi == beta
    # Rm(t) == return on market (usually refers to S&P)
    # Alphai(t) == alpha of particular stock on that day
    
    #For individual stocks	
    Ri(t) = Bi*Rm(t) + Alphai(t)
    
    #CAPM for Portfolios:
    Bp = Sum(Wi * Bi)
    Alphap(t) = Sum(Wi * alphai(t))
    Rp(t) = Bp* Rm(t) + Alphap(t)

def technical_analysis():
    #Technical indicators:
    momentum[t] = price[t]/price[t-n] - 1
    SMA[t] = price[t]/price[t-n:t].mean() - 1
    BB[t] = price[t]-SMA[t]/2*std[t]
    normed = values - mean / values.std()
    
    #Grinolds fundamental law:
    performance = skill * sqrt(breadth)
    SRmulti = SRsingle * sqrt(bets)
    
    #IR is like Sharpe of excess return
    information_ratio = mean(Alphap(t))/std(Alphap(t))
    #IC(info coefficient) is correlation of forecasts to returns
    #BR(Breadth) is number of trading opportunities per year
    IR = IC * sqrt(BR)

In [4]:
def computeBollingerBands(adj_close_df):
    
    #Compute Bollinger Bands
    # 1. Compute rolling mean
    adj_close_df = adj_close_df.set_index(['Date'])
    i = 0
    for each in adj_close_df:
        rm_SPY = get_rolling_mean(adj_close_df[each], window=20)
        # 2. Compute rolling standard deviation
        rstd_SPY = get_rolling_std(adj_close_df[each], window=20)
        # 3. Compute upper and lower bands
        upper_band, lower_band = get_bollinger_bands(rm_SPY, rstd_SPY)

        if i == 0:
            upper_band_df = pd.DataFrame(upper_band)
            lower_band_df = pd.DataFrame(lower_band)
            i = 1
        else:
            next_upper_df = pd.DataFrame(upper_band)
            next_lower_df = pd.DataFrame(lower_band)
            upper_band_df = pd.merge(upper_band_df, next_upper_df, left_index=True, right_index=True)
            lower_band_df = pd.merge(lower_band_df, next_lower_df, left_index=True, right_index=True)
    
    upper_band_df = upper_band_df[20:]
    lower_band_df = lower_band_df[20:]

    return upper_band_df, lower_band_df

def plotBollinger(adj_close_df):
    
    rm_SPY = get_rolling_mean(adj_close_df, window=20)
    rstd_SPY = get_rolling_std(adj_close_df, window=20)
    upper_band, lower_band = get_bollinger_bands(rm_SPY, rstd_SPY)
    
    # Plot raw SPY values, rolling mean and Bollinger Bands
    ax = adj_close_df[20:160].plot(title="Bollinger Bands", label='IBM')
    rm_SPY[20:].plot(label='Rolling mean', ax=ax)
    upper_band[20:].plot(label='upper band', ax=ax)
    lower_band[20:].plot(label='lower band', ax=ax)
    
    # Add axis labels and legend
    ax.set_xlabel("Date")
    ax.set_ylabel("Price")
    ax.legend(loc='upper left')
    plt.show()    
    
def computeAndPlotDailyReturns(adj_close_df):
    #Compute daily returns
    adj_close_df = adj_close_df.set_index(['Date'])
    adj_close_df = adj_close_df
    daily_returns = compute_daily_returns(adj_close_df)
    plot_data(daily_returns, title="Daily returns", ylabel="Daily returns")

In [ ]:
adj_close_df, adj_vol_df = createAdjVolAndCloseDfs(dowJonesDict)
upper_band_df, lower_band_df = computeBollingerBands(adj_close_df)

plotBollinger(adj_close_df['Nike'][:160])
computeAndPlotDailyReturns(adj_close_df[['Date','IBM','Nike','Visa']][0:20])